In [198]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from multiprocessing import Pool
from functools import partial
from copy import deepcopy
import time, os, sys

from sklearn import linear_model, metrics
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

In [199]:
os.chdir("/media/eys/xwj/mywork_B-GEX/")
gene_tpm_raw = pd.read_csv("GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.no_pseudogene", sep="\t", skiprows=2)

samples = pd.read_csv("GTEx_v7_Annotations_SampleAttributesDS.txt", sep="\t")
samples = samples[samples.SMAFRZE == 'RNASEQ']

'''add one column-SUBJID '''
samples.insert(loc=0, column='SUBJID', value =np.repeat(0,samples.shape[0]))
'''SUBJID is made from SAMPID'''
samples=samples.reset_index(drop=True)
for i in range(samples.shape[0]):
    temp=samples.loc[i,'SAMPID'].split('-')
    samples.loc[i,'SUBJID']=temp[0]+'-'+temp[1]
del temp

'''blood and tissue RNAseq samples basic information'''
TISSUE_list = samples.loc[(samples.SMTSD != 'Whole Blood'), ['SMTS','SMTSD']]
TISSUE_list = TISSUE_list.drop_duplicates().reset_index(drop=True)

feature_id = samples[(samples.SMTSD == 'Whole Blood') & (samples.SMAFRZE == 'RNASEQ')].loc[:,['SUBJID','SAMPID']]

In [200]:
gene_tpm_raw["Name"]=[x.split(".")[0] for x in gene_tpm_raw["Name"]] 

gene_tpm= deepcopy(gene_tpm_raw)
gene_tpm.iloc[:, 2:] = np.log(gene_tpm.iloc[:, 2:]+1)

In [201]:
for t in TISSUE_list.index:
    TISSUE = TISSUE_list.SMTSD[t]
    target_id = samples[(samples.SMTSD == TISSUE ) & (samples.SMAFRZE == 'RNASEQ')].loc[:,['SUBJID','SAMPID']]
    sample_overlap = list(set(target_id.SUBJID) & set(feature_id.SUBJID))
    feature_tpm = gene_tpm[["Name", "Description"] + list(feature_id[feature_id.SUBJID.isin(sample_overlap)].SAMPID)]
    target_tpm = gene_tpm[["Name", "Description"] + list(target_id[target_id.SUBJID.isin(sample_overlap)].SAMPID)]
    # fill tissue sample number statistics
    TISSUE_list.loc[t,'Bsample'] = feature_id.shape[0]
    TISSUE_list.loc[t,'Tsample'] = target_id.shape[0]
    TISSUE_list.loc[t,'Bgenes'] = feature_tpm.shape[0]
    TISSUE_list.loc[t,'Tgenes'] = target_tpm.shape[0]
    TISSUE_list.loc[t,'BTsample'] = len(sample_overlap)
    TISSUE_list.loc[t,'TgenesClean'] = target_tpm.loc[target_tpm.all(axis=1)].shape[0]
    TISSUE_list.loc[t,'BgenesClean'] = feature_tpm.loc[feature_tpm.all(axis=1)].shape[0]

TISSUE_list_short = TISSUE_list[(TISSUE_list.BTsample > 100) &
                          (TISSUE_list.SMTSD.values != 'Cells - Transformed fibroblasts') &
                          (TISSUE_list.SMTSD.values != 'Cells - EBV-transformed lymphocytes')].reset_index(drop=True)
TISSUE_list_short.head(2)

,SMTS,SMTSD,Bsample,Tsample,Bgenes,Tgenes,BTsample,TgenesClean,BgenesClean
0,Adipose Tissue,Adipose - Subcutaneous,407.0,442.0,42075.0,42075.0,278.0,15917.0,12121.0
1,Muscle,Muscle - Skeletal,407.0,564.0,42075.0,42075.0,341.0,14006.0,12000.0


In [202]:
gene_tpm=gene_tpm.set_index("Name").drop(columns="Description")

In [203]:
use_feature=[0.10]
use_target=[1.00]
optimal_features=[5, 10,15, 20, 25, 30]
# "randData_cosine"
do_select=["lasso","cosine","randSelect"]
# models=["lsr","bay"]
raw_df=["summary0", "summary2", "train0", "val2", "train_mean","train_std"]
metric_summary=pd.DataFrame(index=TISSUE_list_short.SMTSD, 
                            columns=["mae_mean","rmse_mean","r2_mean", "mae_std","rmse_std","r2_std"])
model_dict = {
    "lsr":  linear_model.LinearRegression(),
    "Lasso": linear_model.Lasso(alpha=0.04, max_iter=1e5),
    "Ridge": linear_model.Ridge(),
    "Bay": linear_model.BayesianRidge()
    }

In [204]:
input_dict={}
for TISSUE in TISSUE_list_short.SMTSD:

    input_dict[TISSUE]={}
    input_dict[TISSUE]["feature"],  input_dict[TISSUE]["target"] ={},{}
    input_dict[TISSUE]["feature"]["train0"]={}
    input_dict[TISSUE]["feature"]["val2"]={}
    input_dict[TISSUE]["target"]["train0"]={}
    input_dict[TISSUE]["target"]["val2"]={}
    
    input_dict[TISSUE]["feature"]["train_mean"]={}
    input_dict[TISSUE]["feature"]["train_std"]={}
    input_dict[TISSUE]["target"]["train_mean"]={}
    input_dict[TISSUE]["target"]["train_std"]={}
    
    target_id = samples[(samples.SMTSD == TISSUE ) & (samples.SMAFRZE == 'RNASEQ')].loc[:,['SUBJID','SAMPID']]
    sample_overlap = list(set(target_id.SUBJID) & set(feature_id.SUBJID))
    feature_tpm = gene_tpm[list(feature_id[feature_id.SUBJID.isin(sample_overlap)].SAMPID)]
    target_tpm = gene_tpm[list(target_id[target_id.SUBJID.isin(sample_overlap)].SAMPID)]
    
#     print(time.ctime(), TISSUE, feature_tpm.shape, target_tpm.shape)
    feature_tpm = feature_tpm.loc[feature_tpm.all(axis=1)] #.reset_index(drop=True)
    target_tpm = target_tpm.loc[target_tpm.all(axis=1)] #.reset_index(drop=True)
    print(time.ctime(), TISSUE, feature_tpm.shape, target_tpm.shape)
    
    # split samples into 90% train+test set and 10% vaildation set
    sample_use, sample_val2=train_test_split(sample_overlap, test_size=0.1, random_state=123, shuffle=True)

    feature_tpm_use = feature_tpm[list(feature_id[feature_id.SUBJID.isin(sample_use)].SAMPID)]
    target_tpm_use = target_tpm[list(target_id[target_id.SUBJID.isin(sample_use)].SAMPID)]

    feature_tpm_val2 = feature_tpm[list(feature_id[feature_id.SUBJID.isin(sample_val2)].SAMPID)]
    target_tpm_val2 = target_tpm[list(target_id[target_id.SUBJID.isin(sample_val2)].SAMPID)]
    
    # compute coefficient of variation with train set ====cv
    for uf in use_feature:
        percent=uf
        
        tissue_tpm=feature_tpm_use
        cutoff = tissue_tpm.shape[0]*percent
        cv=tissue_tpm.std(axis=1)/tissue_tpm.mean(axis=1)
        mask=cv.rank(ascending=False) <= cutoff
        tissue_tpm = tissue_tpm[mask]
        # keep the train_mean, and train_std values before transform data
        train_mean, train_std = tissue_tpm.mean(axis=1), tissue_tpm.std(axis=1)
        
        tissue_tpm = tissue_tpm.transform(lambda x: (x-x.mean())/x.std(), axis=1).round(3)       
        input_dict[TISSUE]["feature"]["train0"][uf]=tissue_tpm
        input_dict[TISSUE]["feature"]["train_mean"][uf]=train_mean
        input_dict[TISSUE]["feature"]["train_std"][uf]=train_std
        
        tissue_tpm=feature_tpm_val2[mask]
        for col in tissue_tpm:
            tissue_tpm.loc[:, col]=(tissue_tpm.loc[:, col]-train_mean)/train_std
        input_dict[TISSUE]["feature"]["val2"][uf]=tissue_tpm
    
    for ut in use_target: 
        percent=ut
        
        tissue_tpm=target_tpm_use
        cutoff = tissue_tpm.shape[0]*percent
        cv=tissue_tpm.std(axis=1)/tissue_tpm.mean(axis=1)
        mask=cv.rank(ascending=False) <= cutoff
        tissue_tpm = tissue_tpm[mask]
        # keep the train_mean, and train_std values before transform data
        train_mean, train_std = tissue_tpm.mean(axis=1), tissue_tpm.std(axis=1)
       
        tissue_tpm = tissue_tpm.transform(lambda x: (x-x.mean())/x.std(), axis=1).round(3)       
        input_dict[TISSUE]["target"]["train0"][ut]=tissue_tpm
        input_dict[TISSUE]["target"]["train_mean"][uf]=train_mean
        input_dict[TISSUE]["target"]["train_std"][uf]=train_std

        tissue_tpm=target_tpm_val2[mask]
        for col in tissue_tpm:
            tissue_tpm.loc[:, col]=(tissue_tpm.loc[:, col]-train_mean)/train_std
        input_dict[TISSUE]["target"]["val2"][ut]=tissue_tpm

    print(time.ctime(), TISSUE, len(input_dict[TISSUE]["feature"]["val2"]), len(input_dict[TISSUE]["target"]["val2"]))        

Fri Apr 10 14:42:05 2020 Adipose - Subcutaneous (12121, 278) (15917, 278)


/home/xwj/soft/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:576: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


Fri Apr 10 14:42:10 2020 Adipose - Subcutaneous 1 1
Fri Apr 10 14:42:10 2020 Muscle - Skeletal (12000, 341) (14006, 341)
Fri Apr 10 14:42:15 2020 Muscle - Skeletal 1 1
Fri Apr 10 14:42:15 2020 Artery - Tibial (12334, 283) (15029, 283)
Fri Apr 10 14:42:19 2020 Artery - Tibial 1 1
Fri Apr 10 14:42:19 2020 Artery - Coronary (13079, 116) (16552, 116)
Fri Apr 10 14:42:24 2020 Artery - Coronary 1 1
Fri Apr 10 14:42:25 2020 Heart - Atrial Appendage (13037, 171) (15636, 171)
Fri Apr 10 14:42:29 2020 Heart - Atrial Appendage 1 1
Fri Apr 10 14:42:30 2020 Adipose - Visceral (Omentum) (12614, 197) (16163, 197)
Fri Apr 10 14:42:34 2020 Adipose - Visceral (Omentum) 1 1
Fri Apr 10 14:42:34 2020 Breast - Mammary Tissue (12602, 173) (16373, 173)
Fri Apr 10 14:42:39 2020 Breast - Mammary Tissue 1 1
Fri Apr 10 14:42:40 2020 Skin - Not Sun Exposed (Suprapubic) (12555, 203) (16639, 203)
Fri Apr 10 14:42:45 2020 Skin - Not Sun Exposed (Suprapubic) 1 1
Fri Apr 10 14:42:45 2020 Adrenal Gland (12779, 121) (168

In [205]:
final={}
for uf in use_feature:
    if uf not in final:
        final[uf]={}
    for ut in use_target:
        if ut not in final[uf]:
            final[uf][ut]={}
        for of in optimal_features[1:2]:
            if of not in final[uf][ut]:
                final[uf][ut][of]={}
            for do in do_select[1:2]:
                if do not in final[uf][ut][of]:
                    final[uf][ut][of][do]={}
                # save feature names in df
                final[uf][ut][of][do]["selected_feature"]={}
                final[uf][ut][of][do]["trained_model"]={}
                
                # save pre_trained model parameters 
                for m in model_dict:
                    if m not in final[uf][ut][of][do]["trained_model"]:
                        final[uf][ut][of][do]["trained_model"][m]={}
                    
                    if m not in final[uf][ut][of][do]:
                        final[uf][ut][of][do][m]={}
                        for rdf in raw_df:
                            if rdf =="summary0" or rdf =="summary2":
                                final[uf][ut][of][do][m][rdf]=deepcopy(metric_summary)
                            else:
                                final[uf][ut][of][do][m][rdf]={}

In [206]:
                        
def worker_return_feature_and_final_models(p):
    
    y_val, y_use=Y_val[p], Y_use[p]
    gene_error=np.zeros(9)

    '''# [2] use cosine similarity way to select topN , only use TRAIN set :  a better feature selection methods??'''
    select=X_use.columns[cos_top[p]]
    X_use_short=X_use[select]
        
    '''fit model with selected feature_and_final'''
    mymodel=model_dict[M].fit(X_use_short, y_use)
#     y_predict = mymodel.predict(X_val_short)

    return select, mymodel

def worker_with_test(p):
    
    y_val, y_use=Y_val[p], Y_use[p]
    gene_error=np.zeros(9)

    '''# [2] use cosine similarity way to select topN , only use TRAIN set :  a better feature selection methods??'''
    select=X_use.columns[cos_top[p]]
    X_val_short, X_use_short=X_val[select], X_use[select]
        
    '''fit model with selected features'''
    '''test error'''
    mymodel=model_dict[M].fit(X_use_short, y_use)
   
    y_predict = mymodel.predict(X_val_short)
    gene_error[6]= metrics.mean_absolute_error(y_val, y_predict)
    gene_error[7]= np.sqrt(metrics.mean_squared_error(y_val, y_predict))
    gene_error[8]= np.corrcoef(y_val.astype(float), y_predict)[1,0]                 
    # get back result of train，test, validat in a long row vector
    return gene_error

'''how many CPUs to use (4 cpu ~ 30 mins)'''
CORE=4

'''set topN: best N features using train set?'''
topN=10
uf, ut =0.10, 1.00
print(optimal_features, CORE, topN, uf, ut)

[5, 10, 15, 20, 25, 30] 4 10 0.1 1.0


In [207]:
for TISSUE in list(input_dict.keys())[:2]:
# for TISSUE in input_dict:

    feature_tpm, target_tpm = input_dict[TISSUE]["feature"]["train0"][uf], input_dict[TISSUE]["target"]["train0"][ut]
    feature_tpm_val, target_tpm_val = input_dict[TISSUE]["feature"]["val2"][uf], input_dict[TISSUE]["target"]["val2"][ut]
    
#     feature_tpm, target_tpm = input_dict_rand[TISSUE]["feature"], input_dict_rand[TISSUE]["target"]
    print(time.ctime(), TISSUE, feature_tpm.shape, target_tpm.shape)
    X_use, Y_use, X_val, Y_val =feature_tpm.transpose(), target_tpm.transpose(), feature_tpm_val.transpose(), target_tpm_val.transpose()
    
    for M in list(model_dict.keys())[3:]:
        # take evaluation results df (valid OR test error ) and df0 (train error)
        for ds in do_select[1:2]: #do_select=["lasso","cosine","randSelect"]        
            
            for topN in optimal_features[1:2]:
                print(time.ctime(), TISSUE, ds, topN, M)

                cos=pd.DataFrame(data=metrics.pairwise.cosine_similarity(X=X_use.transpose(), Y=Y_use.transpose(), dense_output=True))
                cos_top=(abs(cos).rank(axis=0, ascending=False) <= topN)
                cos_top.columns = target_tpm.index
                
                final[uf][ut][topN][ds]["selected_feature"][TISSUE]=pd.DataFrame(index=Y_use.columns, columns=range(topN))
                final[uf][ut][topN][ds]["trained_model"][M][TISSUE]={}
                
                '''save final features and model parameters'''
                for p in Y_use.columns:
                    # select, mymodel.best_estimator_ 
                    final[uf][ut][topN][ds]["selected_feature"][TISSUE].loc[p,:], \
                    final[uf][ut][topN][ds]["trained_model"][M][TISSUE][p] \
                    =worker_return_feature_and_final_models(p) 
                
                '''save test error'''
                with Pool(CORE) as cpu_pool:
                    gene_error_in_rows = cpu_pool.map(partial(worker_with_test), Y_use.columns)

                gene_error=pd.DataFrame(data=gene_error_in_rows, index=Y_use.columns)                  
                metric_df2=pd.DataFrame(index=Y_use.columns, columns=["mae","rmse","r2"])
                metric_df2["mae"], metric_df2["rmse"], metric_df2["r2"]=gene_error[6],gene_error[7], gene_error[8]
                final[uf][ut][topN][ds][M]["val2"][TISSUE]=metric_df2
                final[uf][ut][topN][ds][M]["summary2"].loc[TISSUE,:]=list(metric_df2.mean(axis=0))+ list(metric_df2.std(axis=0)) 
                

Fri Apr 10 14:44:24 2020 Adipose - Subcutaneous (1212, 250) (15917, 250)
Fri Apr 10 14:44:24 2020 Adipose - Subcutaneous cosine 10 Bay
Fri Apr 10 14:46:01 2020 Muscle - Skeletal (1200, 306) (14006, 306)
Fri Apr 10 14:46:01 2020 Muscle - Skeletal cosine 10 Bay


In [208]:
# BGEX predict with pretrained models
TISSUE="Adipose - Subcutaneous"
ds="cosine"
topN, uf, ut =10, 0.10, 1.00
M="Bay" # BayesianRidge regression

feature_std=input_dict[TISSUE]["feature"]["train_std"][uf]
feature_mean=input_dict[TISSUE]["feature"]["train_mean"][uf]

target_std=input_dict[TISSUE]["target"]["train_std"][uf]
target_mean=input_dict[TISSUE]["target"]["train_mean"][uf]

# input a blood gene expression profile in TPM value.
# Here use TWO training sample as tutorial example 
# step 1. take releveant features from the raw blood profile
raw_blood_profile_in_TPM=gene_tpm_raw.loc[:, ["Name","GTEX-111YS-0006-SM-5NQBE", "GTEX-1128S-0005-SM-5P9HI"]].set_index("Name")
new_blood_profile = raw_blood_profile_in_TPM.reindex(feature_mean.index)

# step 2. normalize by predefined mean and std
for col in new_blood_profile:
    new_blood_profile.loc[:, col] = (np.log(new_blood_profile.loc[:, col] +1) - feature_mean)/feature_std

# step 3. use pre-trained model to predict gene expression of new sample's tissue
predicted_tissue_profile=pd.DataFrame(index=target_mean.index, columns=new_blood_profile.columns) 

total=len(target_mean)
# for p in target_mean.index:
for i in range(total):
    p=target_mean.index[i]
    # select, mymodel.best_estimator_ 
    selected=final[uf][ut][topN][ds]["selected_feature"][TISSUE].loc[p,:]
    mymodel=final[uf][ut][topN][ds]["trained_model"][M][TISSUE][p]
    X_val_short=new_blood_profile.transpose()[selected]
    predicted_tissue_profile.loc[p,:] = mymodel.predict(X_val_short) 
    if (i % 1000==0) :
        print(i,"/", total)
    
# step 4. revert predicted value to TPM
for col in predicted_tissue_profile:
    predicted_tissue_profile.loc[:, col] = np.e**(predicted_tissue_profile.loc[:, col] * target_std + target_mean) -1

# step 5. filter by empirical predictability mark
thres_mae, thres_r2 = 0.7, 0.3
d0=final[uf][ut][topN][ds][M]["val2"][TISSUE]
predictable_gene=(d0["mae"] < thres_mae) & (d0["r2"] > thres_r2) 
predicted_tissue_profile_short=predicted_tissue_profile[predictable_gene]

0 / 15917
1000 / 15917
2000 / 15917
3000 / 15917
4000 / 15917
5000 / 15917
6000 / 15917
7000 / 15917
8000 / 15917
9000 / 15917
10000 / 15917
11000 / 15917
12000 / 15917
13000 / 15917
14000 / 15917
15000 / 15917
